In [1]:
import os
from dotenv import load_dotenv
import boto3
import json
import base64

from aws_summary_template import SUMMARY_TEMPLATE

load_dotenv()

True

In [2]:
AWS_REGION_NAME = 'us-west-2'
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime.html
bedrock = boto3.client(
    service_name='bedrock-runtime',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=AWS_REGION_NAME
)

In [3]:
# trouble-shooting: use a different client with service_name 'bedrock', not 'bedrock-runtime'
# https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock.html

# client = boto3.client(
#     service_name='bedrock',
#     aws_access_key_id=aws_access_key_id,
#     aws_secret_access_key=aws_secret_access_key,
#     region_name=AWS_REGION_NAME
# )

# summ = client.list_foundation_models()['modelSummaries']
# [model for model in summ if 'Sonnet' in model['modelName']]

In [4]:
# [m for m in dir(bedrock) if not m.startswith('_')]
# help(bedrock.converse)

In [5]:
if False:
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            { "text": "How are you today?" } 
        ],
    }

    message_list.append(initial_message)

    response = bedrock.converse(
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2000,
            "temperature": 0
        },
    )

    response_message = response['output']['message']
    print(json.dumps(response_message, indent=4))

In [6]:
tool_list = [{
    "toolSpec": SUMMARY_TEMPLATE
}]


In [7]:
filename = "IT Consulting in the US.pdf"
content = open(filename, 'rb').read()
encoded = base64.b64encode(content)

prompt = """
You are an expert in extracting market and financial data from documents.
Extract essential data from the enclosed document.

Split paragraphs into sentences preceded by the subject. For example:
"Revenue Growth: Revenue has grown at a CAGR of 2.8% to $692.9 billion over the past five years."

Return the result in JSON format. Only return JSON and nothing else.
"""

initial_message = {
    "role": "user",
    "content": [
        {
            "text": prompt,
        },
        {
            "document": {
                "format": "pdf",
                "name": 'document',
                "source": {
                    "bytes": content
                }
            }
        }
    ],
}

response = bedrock.converse(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    # modelId="meta.llama3-1-405b-instruct-v1:0",
    messages=[initial_message],
    inferenceConfig={
        "maxTokens": 4000,
        "temperature": 0
    },
    toolConfig={
        "tools": tool_list,
        "toolChoice": {
            "tool": {
                "name": "summarize_document"
            }
        }
    }
)

In [8]:
response_message = response['output']['message']
# print(json.dumps(response_message, indent=4))

core_response = response_message['content'][0]['toolUse']["input"]
print(json.dumps(core_response, indent=2))

{
  "key_statistics": "Revenue: $710,683 million in 2024 (projected)\nRevenue growth (CAGR): \n  2002-2024: 3.93%\n  2024-2029 (projected): 2.36%\nEnterprises: 496,630 in 2024 (projected)\nEnterprise growth (CAGR):\n  2002-2024: 1.33% \n  2024-2029 (projected): 2.10%\nEstablishments: 518,365 in 2024 (projected)  \nEstablishment growth (CAGR):\n  2002-2024: 1.40%\n  2024-2029 (projected): 2.11%  \nEmployment: 2,542,152 in 2024 (projected)\nEmployment growth (CAGR):\n  2002-2024: 2.86%\n  2024-2029 (projected): 2.32%\nWages: $305,094 million in 2024 (projected)\nWage growth (CAGR):\n  2002-2024: 3.97%\n  2024-2029 (projected): 2.33%\nIndustry Value Added (IVA): $356,003 million in 2024 (projected)\nIVA growth (CAGR): \n  2002-2024: 4.11%\n  2024-2029 (projected): 2.34%",
  "executive_summary": "The IT Consulting industry has experienced revenue growth due to increasing corporate profit and demand for cloud computing and data analytics services. Major players have acquired smaller compani

In [9]:
print(core_response['key_statistics'])

Revenue: $710,683 million in 2024 (projected)
Revenue growth (CAGR): 
  2002-2024: 3.93%
  2024-2029 (projected): 2.36%
Enterprises: 496,630 in 2024 (projected)
Enterprise growth (CAGR):
  2002-2024: 1.33% 
  2024-2029 (projected): 2.10%
Establishments: 518,365 in 2024 (projected)  
Establishment growth (CAGR):
  2002-2024: 1.40%
  2024-2029 (projected): 2.11%  
Employment: 2,542,152 in 2024 (projected)
Employment growth (CAGR):
  2002-2024: 2.86%
  2024-2029 (projected): 2.32%
Wages: $305,094 million in 2024 (projected)
Wage growth (CAGR):
  2002-2024: 3.97%
  2024-2029 (projected): 2.33%
Industry Value Added (IVA): $356,003 million in 2024 (projected)
IVA growth (CAGR): 
  2002-2024: 4.11%
  2024-2029 (projected): 2.34%
